In [1]:
import requests
from bs4 import BeautifulSoup
response = requests.get("https://www.themoviedb.org/tv")
base_url = "http://quotes.toscrape.com/"

def parse_quote_page(url):
    """ Used to parse the parse the webpage and return a beautiful soup object"""
    quote_url = url
    response = requests.get(quote_url)
    if response.status_code != 200:
        print('Status code:', response.status_code)
        raise Exception('Failed to fetch web page ' + quote_url)
    return BeautifulSoup(response.text)

In [2]:
import requests
from bs4 import BeautifulSoup
needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
response = requests.get("https://www.themoviedb.org/tv", headers = needed_headers)

In [3]:
response.status_code

200

In [4]:
dwn_content = response.text
len(dwn_content)

200997

In [5]:
dwn_content[:500]

'<!DOCTYPE html>\n<html lang="en" class="no-js">\n  <head>\n    <title>Popular TV Shows &#8212; The Movie Database (TMDB)</title>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n    <meta http-equiv="cleartype" content="on">\n    <meta charset="utf-8">\n    <meta name="keywords" content="Movies, TV Shows, Streaming, Reviews, API, Actors, Actresses, Photos, User Ratings, Synopsis, Trailers, Teasers, Credits, Cast">\n    <meta name="mobile-web-app-capable" content="yes">\n    <meta name="apple'

In [6]:
test_doc = BeautifulSoup(response.text, 'html.parser')
type(test_doc)

bs4.BeautifulSoup

In [7]:
test_doc.find('title')

<title>Popular TV Shows — The Movie Database (TMDB)</title>

In [8]:
test_doc.find('img')

<img alt="The Movie Database (TMDB)" height="20" src="/assets/2/v4/logos/v2/blue_short-8e7b30f73a4020692ccca9c88bafe5dcb6f8a62a4c6bc55cd9ba82bb2cd95f6c.svg" width="154"/>

In [9]:
def get_page_content(url):
    # In this case , we are going to give request.get function headers to avoid the Status code Error 403

    get_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response_page = requests.get(url, headers = get_headers )
    # we are going to raise exception here if status code gives any value other than 200.
    if not response_page.ok:
        raise Exception ("Failed to request the data. Status Code:- {}".format(response_page.status_code))
    else:
        page_content = response_page.text
        doc_page = BeautifulSoup(page_content, "html.parser")
        return doc_page

In [10]:
popular_shows_url = "https://www.themoviedb.org/tv"
doc = get_page_content(popular_shows_url)

In [11]:
doc.title.text

'Popular TV Shows — The Movie Database (TMDB)'

In [12]:
doc.find_all('div', {'class': 'card style_1'})[0].h2.text

'Love Island'

In [13]:
doc.find_all('div', {'class': 'user_score_chart'})[0]['data-percent']


'76.0'

In [14]:
def empty_dict():
    scraped_dict = {  
    'Title': [],
    'User_rating': [], 
    'Release_date':[], 
    'Current_season': [],
    'Current_season_Episodes': [], 
    'Tagline': [],
    'Genre': [],
    'Cast': []   
    }
    return scraped_dict

In [15]:
def user_score_info(tag_user_score, i, scraped_dict):
    if tag_user_score[i]['data-percent'] == '0':
        scraped_dict['User_rating'].append('Not rated yet')
    else:
        scraped_dict['User_rating'].append(tag_user_score[i]['data-percent'])


In [16]:
doc.find_all('div', {'class': 'card style_1'})[0].h2.a['href']

'/tv/90521'

In [17]:
def get_show_info(doc_page):
    base_link_1 = "https://www.themoviedb.org"
    tag_title = tag_premired_date = tag_shows_page = doc_page.find_all('div', {'class': 'card style_1'})
    tag_user_score = doc_page.find_all('div', {"user_score_chart"}) 
    
    doc_2_list = []
    for link in tag_shows_page:
        # here we are creating the list of all the individual pages of the shows which will come handy in other functions. 
        doc_2_list.append(get_page_content("https://www.themoviedb.org" + link.h2.a['href']))
       # we are going to have the function to return the list of all the information as elements. 
    return tag_title, tag_user_score, doc_2_list
# lets see if the function returns the list of the information we tried to get earlier. 
len(get_show_info(doc))

3

In [18]:
doc_2 = get_page_content("https://www.themoviedb.org/tv/91363")


In [19]:
tag_genre = doc_2.find('span', {"class": "genres"})
tag_genre_list = tag_genre.find_all('a')
check_genre =[]
for tag in tag_genre_list:
    check_genre.append(tag.text)
check_genre
['Animation', 'Action & Adventure', 'Sci-Fi & Fantasy']

['Animation', 'Action & Adventure', 'Sci-Fi & Fantasy']

In [20]:
def get_genres(doc2_page, i):
    genres_tags = doc2_page[i].find('span', {"class": "genres"}).find_all('a')
    check_genre =[]
    
    for tag in genres_tags:
        check_genre.append(tag.text)
    return check_genre

In [21]:
tag_tagline = doc_2.find('h3',{"class": 'tagline'})

def tagline_info(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('h3',{"class": 'tagline'}):
        scraped_dict['Tagline'].append(doc_2_list[i].find('h3',{"class": 'tagline'}).text)
    else:
        scraped_dict['Tagline'].append("No Tagline")

In [22]:
def get_show_cast(doc2_page, i):
    cast_tags = doc2_page[i].find_all('li', {'class': 'card'})
    cast_lis = []
    
    for t in cast_tags:
         cast_lis.append(t.p.text)

    return cast_lis

In [23]:
tag_episodes = doc_2.find_all('div' , {'class': 'flex'})
# extracing current season from h2 tag under class flex.
tag_episodes[1].h2.text 

'Season 1'

In [24]:
print('2021 | 9 Episodes'[7:])

tag_episodes[1].h4.text[7:]

9 Episodes


'9 Episodes'

In [25]:
import pandas as pd

def get_show_details(t_title, t_user_score, docs_2_list):
    # excuting a function here that empties the dictionary every time the function is called.
    scraped_dict =  empty_dict()
    for i in range (0, len(t_title)):
        scraped_dict['Title'].append(t_title[i].h2.text)
        user_score_info(t_user_score, i, scraped_dict)    
        scraped_dict['Release_date'].append(t_title[i].p.text)
        scraped_dict['Current_season'].append(docs_2_list[i].find_all('div' , {'class': 'flex'})[1].h2.text)
        scraped_dict['Current_season_Episodes'].append(docs_2_list[i].find_all('div' , {'class': 'flex'})[1].h4.text[7:])
        tagline_info(docs_2_list, i, scraped_dict)  
        scraped_dict['Genre'].append(get_genres(docs_2_list, i))        
        scraped_dict['Cast'].append(get_show_cast(docs_2_list, i))
        
    return pd.DataFrame(scraped_dict)

In [26]:
x = get_show_details(tag_title, tag_user_score, doc_2_list)
x.to_csv('check.csv')
pd.read_csv('check.csv',index_col=[0])

NameError: name 'tag_title' is not defined

In [27]:
import os
base_link = "https://www.themoviedb.org/tv"

# 'i' here means the number of page we want to extract
def create_page_df( i, dataframe_list):
    os.makedirs('shows-data', exist_ok = True)
    next_url = base_link + '?page={}'.format(i)
    doc_top = get_page_content(next_url)
    name_tag, viewer_score_tag, doc_2_lis = get_show_info(doc_top)
    print('scraping page {} :- {}'.format(i, next_url))
    dataframe_data = get_show_details(name_tag, viewer_score_tag, doc_2_lis)
    dataframe_data.to_csv("shows-data/shows-page-{}.csv".format(i) , index = None)
    print(" ---> a CSV file with name shows-page-{}.csv has been created".format(i))
    dataframe_list.append(dataframe_data)


In [28]:
test_list = []
create_page_df(9 , test_list)

scraping page 9 :- https://www.themoviedb.org/tv?page=9
 ---> a CSV file with name shows-page-9.csv has been created


In [29]:
import pandas as pd
base_link = "https://www.themoviedb.org/tv"

def scrape_top_200_shows(base_link):
    dataframe_list = []
    # we are going to keep range up to 11 because we just need up to 200 TV shows for now. 
    for i in range(1,11):
        create_page_df(i, dataframe_list)
    # here we are using concat function so that we can merge the each dataframe that we got from the each page.    
    total_dataframe = pd.concat(dataframe_list, ignore_index = True)
    
    # with the simple command of to_csv() we can create a csv file of all the pages we extracted.
    csv_complete =  total_dataframe.to_csv('shows-data/Total-dataframe.csv', index= None)
    print(" \n a CSV file named Total-dataframe.csv with all the scraped shows has been created")

In [30]:
scrape_top_200_shows(base_link)

scraping page 1 :- https://www.themoviedb.org/tv?page=1
 ---> a CSV file with name shows-page-1.csv has been created
scraping page 2 :- https://www.themoviedb.org/tv?page=2
 ---> a CSV file with name shows-page-2.csv has been created
scraping page 3 :- https://www.themoviedb.org/tv?page=3
 ---> a CSV file with name shows-page-3.csv has been created
scraping page 4 :- https://www.themoviedb.org/tv?page=4
 ---> a CSV file with name shows-page-4.csv has been created
scraping page 5 :- https://www.themoviedb.org/tv?page=5
 ---> a CSV file with name shows-page-5.csv has been created
scraping page 6 :- https://www.themoviedb.org/tv?page=6
 ---> a CSV file with name shows-page-6.csv has been created
scraping page 7 :- https://www.themoviedb.org/tv?page=7
 ---> a CSV file with name shows-page-7.csv has been created
scraping page 8 :- https://www.themoviedb.org/tv?page=8
 ---> a CSV file with name shows-page-8.csv has been created
scraping page 9 :- https://www.themoviedb.org/tv?page=9
 ---> a 

Top quotes for tag "love" written to file quotes.csv


'quotes-scraped.csv'

Top quotes for tag "inspirational" written to file quotes.csv


'quotes-scraped.csv'

Top quotes for tag "life" written to file quotes.csv


'quotes-scraped.csv'

Top quotes for tag "humor" written to file quotes.csv


'quotes-scraped.csv'